<a href="https://colab.research.google.com/github/stanislavsvystovych/CV-labs/blob/main/age_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! [ ! -d "/content/dataset/UTKFace" ] && unzip "/content/drive/MyDrive/Datasets/utkface.zip" -d dataset

In [ ]:
dataset_folder_name = '/content/dataset/UTKFace'

IMG_WIDTH = IMG_HEIGHT = 200

In [ ]:
import glob
import os
import random

filenames = glob.glob(os.path.join(dataset_folder_name, "*.%s" % 'jpg'))
random.shuffle(filenames)

In [ ]:
from tensorflow.keras.utils import Sequence
import math
import numpy as np
from PIL import Image

class UTKFaceSequence(Sequence):

    def __init__(self, filenames, batch_size, shuffle=True):
        self.filenames = filenames
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.filenames) / self.batch_size)

    def on_epoch_end(self):
        if self.shuffle == True:
            random.shuffle(self.filenames)

    def __getitem__(self, idx):
        filenames = self.filenames[idx * self.batch_size: (idx + 1) * self.batch_size]
        x_data = []
        y_data = []
        for file in filenames:
            x, y = self.parse_info_from_file(file)
            x_data.append(x)
            y_data.append(y)
        x_data = np.array(x_data)
        y_data = np.array(y_data)
        return x_data, y_data
    
    def get_labels(self):
        labels = []
        for filename in self.filenames:
            filename = os.path.split(filename)[1]
            filename = os.path.splitext(filename)[0]
            age = filename.split('_')[0]
            labels.append(int(age))
        return labels

    @staticmethod
    def parse_info_from_file(img_path):
        img = Image.open(img_path)
        img = img.resize((IMG_WIDTH, IMG_HEIGHT))
  
        filename = os.path.split(img_path)[1]
        filename = os.path.splitext(filename)[0]
        
        # Normalization
        age = float(filename.split('_')[0]) / 116.0
        img = np.array(img) / 255.0

        return img, age

# Defining CNN-model

In [ ]:
import tensorflow as tf

leaky_relu_negative_slope = 0.2

inputs = tf.keras.layers.Input(shape = [IMG_WIDTH] + [IMG_HEIGHT] + [ 3 ])

x = tf.keras.layers.Conv2D(16, kernel_size=(3, 3), strides=1, use_bias=False, 
                           kernel_initializer=tf.keras.initializers.HeNormal(), 
                           kernel_regularizer=tf.keras.regularizers.L2(1e-5))(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU(leaky_relu_negative_slope)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1, use_bias=False,
                           kernel_initializer=tf.keras.initializers.HeNormal(), 
                           kernel_regularizer=tf.keras.regularizers.L2(1e-5))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU(leaky_relu_negative_slope)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1, use_bias=False,
                           kernel_initializer=tf.keras.initializers.HeNormal(), 
                           kernel_regularizer=tf.keras.regularizers.L2(1e-5))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU(leaky_relu_negative_slope)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1, use_bias=False,
                           kernel_initializer=tf.keras.initializers.HeNormal(), 
                           kernel_regularizer=tf.keras.regularizers.L2(1e-5))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU(leaky_relu_negative_slope)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1, use_bias=False,
                           kernel_initializer=tf.keras.initializers.HeNormal(), 
                           kernel_regularizer=tf.keras.regularizers.L2(1e-5))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU(leaky_relu_negative_slope)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1, use_bias=False,
                           kernel_initializer=tf.keras.initializers.HeNormal(), 
                           kernel_regularizer=tf.keras.regularizers.L2(1e-5))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU(leaky_relu_negative_slope)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(256, kernel_regularizer=tf.keras.regularizers.L2(0.1), bias_regularizer=tf.keras.regularizers.L2(0.1))(x)
x = tf.keras.layers.LeakyReLU(alpha=leaky_relu_negative_slope)(x)
x = tf.keras.layers.Dropout(0.6)(x)

x = tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.L2(0.1), bias_regularizer=tf.keras.regularizers.L2(0.1))(x)
x = tf.keras.layers.LeakyReLU(alpha=leaky_relu_negative_slope)(x)
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Dense(32, kernel_regularizer=tf.keras.regularizers.L2(0.1), bias_regularizer=tf.keras.regularizers.L2(0.1))(x)
x = tf.keras.layers.LeakyReLU(alpha=leaky_relu_negative_slope)(x)
x = tf.keras.layers.Dropout(0.2)(x)

outputs = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.models.Model(inputs, outputs)

model.summary()
# tf.keras.utils.plot_model( model , to_file='age_model_architecture.png', show_shapes=True)

# Compiling the model

In [ ]:
num_epochs = 50
batch_size = 128

filepath = 'saved_models/age_recognition_{epoch:02d}_{val_mae:.4f}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, save_best_only=True, save_weights_only=False, monitor='val_mae')
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)

model.compile( 
    loss=tf.keras.losses.mean_absolute_error,
    optimizer=tf.keras.optimizers.Adam(), 
    metrics=['mae', 'mse'])

# Training the model


In [ ]:
!rm -rf 'saved_models'
!rm -rf 'age_recognition.h5'

TRAIN_TEST_SPLIT = 0.8
train_num_samples = int(len(filenames) * TRAIN_TEST_SPLIT)

train_data_generator = UTKFaceSequence(filenames[:train_num_samples], batch_size, shuffle=True)
test_data_generator = UTKFaceSequence(filenames[train_num_samples:], batch_size, shuffle=False)

history = model.fit( 
    x=train_data_generator,
    epochs=num_epochs,
    validation_data=test_data_generator, 
    callbacks=[model_checkpoint_callback, early_stopping_callback])

# Saving the model

In [ ]:
model.save('age_recognition.h5')

# Evaluating the model

In [ ]:
model = tf.keras.models.load_model('age_recognition.h5')
model.evaluate(test_data_generator)

# Visualization of the results




In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 5))
plt.plot(history.history['mae'], color='red', label='Training')
# plt.plot(history.history['val_mae'], color='green', label='Validation')

plt.title("MAE graph")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(loc="upper left")

plt.show()

In [ ]:
plt.figure(figsize=(6, 5))
plt.plot(history.history['mse'], color='red', label='Training')
# plt.plot(history.history['val_mse'], color='green', label='Validation')

plt.title("MSE graph")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(loc="upper left")

plt.show()

In [ ]:
fig = plt.figure(figsize=(10 , 18))
x, y = test_data_generator.__getitem__(2)

for i, (image, label) in enumerate(zip(x[:12], y[:12])):
    fig.add_subplot(5, 3, i + 1)
    plt.imshow(image)
    prediction = round(model.predict(np.expand_dims(image , 0))[0][0] * 116)
    plt.axis('off')
    plt.title(f'Predicted age : {prediction}\n Actual age : {round(label * 116)}')

plt.show()